In [10]:
import pandas as pd

url = 'https://hl7.org/fhir/R4/valueset-practitioner-role.html'
title= 'practitioner-role'
tables = pd.read_html(url, attrs={"class": "codes"})
codigo_table = tables[0]


In [11]:
codigo_table.head()

,0,1,2,3
0,Code,System,Display,Definition
1,doctor,http://terminology.hl7.org/CodeSystem/practiti...,Doctor,A qualified/registered medical practitioner
2,nurse,http://terminology.hl7.org/CodeSystem/practiti...,Nurse,A practitioner with nursing experience that ma...
3,pharmacist,http://terminology.hl7.org/CodeSystem/practiti...,Pharmacist,A qualified/registered/licensed pharmacist
4,researcher,http://terminology.hl7.org/CodeSystem/practiti...,Researcher,A practitioner that may perform research


In [12]:
from pandas import DataFrame

def convertir_a_valueset(tabla: DataFrame, uri: str, title: str):
    value_set = {
        "resourceType": "ValueSet",
        "id": "230",
        "status": "active",
        "url": uri,
        "title": title,
        "publisher": "Ignacio Martínez",
        "compose": {
            "include": []
        }
    }
    
    sistemas = set()  # Conjunto para mantener un registro de los sistemas únicos
    
    # Iterar sobre cada fila de la tabla
    for index, fila in tabla.iterrows():
        code, system, display, definition = fila
        
        # Si hay más de un sistema, dividirlos y procesar individualmente
        sistemas_lista = system.split(",")
        
        for sys in sistemas_lista:
            sys = sys.strip()  # Eliminar espacios en blanco adicionales
            
            # Verificar si el sistema ya ha sido incluido
            if sys not in sistemas:
                concepto_sistema = {
                    "system": sys,
                    "concept": []
                }
                sistemas.add(sys)
                value_set["compose"]["include"].append(concepto_sistema)
            else:
                # Buscar el concepto existente en el sistema actual
                concepto_sistema = next((s for s in value_set["compose"]["include"] if s["system"] == sys), None)
            
            # Agregar el concepto al sistema correspondiente
            concept = {
                "code": code,
                "display": display
            }
            concepto_sistema["concept"].append(concept)
    
    return value_set


In [13]:
value_set_json = convertir_a_valueset(codigo_table, url, title)

In [14]:
import json
print(json.dumps(value_set_json, indent=2))

{
  "resourceType": "ValueSet",
  "id": "230",
  "status": "active",
  "url": "https://hl7.org/fhir/R4/valueset-practitioner-role.html",
  "title": "practitioner-role",
  "publisher": "Ignacio Mart\u00ednez",
  "compose": {
    "include": [
      {
        "system": "System",
        "concept": [
          {
            "code": "Code",
            "display": "Display"
          }
        ]
      },
      {
        "system": "http://terminology.hl7.org/CodeSystem/practitioner-role",
        "concept": [
          {
            "code": "doctor",
            "display": "Doctor"
          },
          {
            "code": "nurse",
            "display": "Nurse"
          },
          {
            "code": "pharmacist",
            "display": "Pharmacist"
          },
          {
            "code": "researcher",
            "display": "Researcher"
          },
          {
            "code": "teacher",
            "display": "Teacher/educator"
          },
          {
            "code": 

In [19]:
with open(f"src/hl7-value-sets/{title}.json", "x") as archivo:
    json.dump(value_set_json, archivo, indent=2)